In [2]:
import os
import cv2
import dlib
import numpy as np
import argparse
from wide_resnet import WideResNet
#import vgg16


def draw_label(image, point, label, font=cv2.FONT_HERSHEY_SIMPLEX,
               font_scale=1, thickness=2):
    size = cv2.getTextSize(label, font, font_scale, thickness)[0]
    x, y = point
    cv2.rectangle(image, (x, y - size[1]), (x + size[0], y), (255, 0, 0), cv2.FILLED)
    cv2.putText(image, label, point, font, font_scale, (255, 255, 255), thickness)


def main():
    
    depth =16 
    k = 8
    

    
    weight_file = os.path.join("pretrained_models", "weights.18-4.06.hdf5")

    # for face detection
    detector = dlib.get_frontal_face_detector()

    # load model and weights
    img_size = 64
    model = WideResNet(img_size, depth=depth, k=k)()
    model.load_weights(weight_file)

    # capture video
    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

    while True:
        # get video frame
        ret, img = cap.read()

        if not ret:
            print("error: failed to capture image")
            return -1

        input_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_h,img_w, _ = np.shape(input_img)

        # detect faces using dlib detector
        detected = detector(input_img, 1)
        faces = np.empty((len(detected), img_size, img_size, 3))

        for i, d in enumerate(detected):
            x1, y1, x2, y2, w, h = d.left(), d.top(), d.right() + 1, d.bottom() + 1, d.width(), d.height()
            xw1 = max(int(x1 - 0.4 * w), 0)
            yw1 = max(int(y1 - 0.4 * h), 0)
            xw2 = min(int(x2 + 0.4 * w), img_w - 1)
            yw2 = min(int(y2 + 0.4 * h), img_h - 1)
            cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)
            # cv2.rectangle(img, (xw1, yw1), (xw2, yw2), (255, 0, 0), 2)
            faces[i,:,:,:] = cv2.resize(img[yw1:yw2 + 1, xw1:xw2 + 1, :], (img_size, img_size))

        if len(detected) > 0:
            # predict ages and genders of the detected faces
            results = model.predict(faces)
            #print(results)
            predicted_genders = results[0]
            #print(len(results[1]))
            ages = np.arange(0, 101).reshape(101, 1)
            #print('d',dot(ages))
            predicted_ages = results[1].dot(ages).flatten()
            #print(predicted_ages)
            #print(predicted_genders)
            age_predict=(predicted_ages[i])
            if age_predict>=10 and age_predict<=15:
                s="10-15"
            elif age_predict>15 and age_predict<=20:
                s="15-20"
            elif age_predict>20 and age_predict<=25:
                s="20-25" 
            elif age_predict>25 and age_predict<=32:
                s="25-32"
            elif age_predict>32 and age_predict<=40:
                s="32-40" 
            elif age_predict>40 and age_predict<=50:
                s="40-50"  
            elif age_predict>50 and age_predict<=65:
                s="50-65"  
            elif age_predict>65 and age_predict<=75:
                s="65-75" 
            elif age_predict>75 and age_predict<=90:
                s="75-90" 
            elif age_predict>0 and age_predict<=1:
                s="0-1" 
            elif age_predict>1 and age_predict<=4:
                s="2-4" 
            elif age_predict>4 and age_predict<=6:
                s="4-6"
            elif age_predict>6 and age_predict<=9:
                s="7-9"    
        # draw results
            #print(s)
        for i, d in enumerate(detected):
            label = "{}, {}".format(s,
                                    "F" if predicted_genders[i][0] > 0.3 else "M")
            draw_label(img, (d.left(), d.top()), label)

        cv2.imshow("result", img)
        key = cv2.waitKey(30)

        if key == 27:
            break


if __name__ == '__main__':
    main()


1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


KeyboardInterrupt: 